In [7]:
import numpy as np
import minterpy as mp
from minterpy.extras.regression import *
from matplotlib import pyplot as plt

import torch
import torchvision
from torchvision import transforms, datasets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.neighbors import NearestNeighbors

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

from matplotlib.colors import Normalize

In [8]:
def getFashionMNIST(batch_size = 10):
    fashionmnist_data = torchvision.datasets.FashionMNIST(download=True, root = 'data/fashionmnist', transform = 
                                                                                 transforms.Compose([transforms.Resize(32),
                                                                                 transforms.ToTensor(), 
                                                                                 transforms.Lambda(lambda x: x.repeat(1, 1, 1))
                                                                                 ]))

    fashionmnist_data_test = torchvision.datasets.FashionMNIST(download=True, root = 'data/fashionmnist', train=False, transform = 
                                                                                 transforms.Compose([transforms.Resize(32),
                                                                                 transforms.ToTensor(), 
                                                                                 transforms.Lambda(lambda x: x.repeat(1, 1, 1))
                                                                                 ]))

    train_loader = torch.utils.data.DataLoader(fashionmnist_data,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=16)

    test_loader = torch.utils.data.DataLoader(fashionmnist_data_test,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=16)

    return train_loader, test_loader



def getDataset(dataset = "MNIST", batch_size = 10):
    if(dataset == "MNIST"):
        train_loader, test_loader = getMNIST(batch_size)
        noChannels,dx, dy = train_loader.dataset.__getitem__(1)[0].shape
    elif(dataset == "FashionMNIST"):
        train_loader, test_loader = getFashionMNIST(batch_size)
        noChannels, dx, dy = train_loader.dataset.__getitem__(1)[0].shape
    elif(dataset == "Cifar10"):
        train_loader, test_loader = getCifar10(batch_size)
        noChannels, dx, dy = train_loader.dataset.__getitem__(1)[0].shape
        """
    elif(dataset == "cityscapes"):
        train_loader, test_loader = getcityscapes(batch_size)
        noChannels, dx, dy = train_loader.dataset.__getitem__(1)[0].shape
        """
    else:
        return None, None, None, None, None    
        
    return train_loader, test_loader, noChannels, dx, dy

In [9]:
train_loader, test_loader, noChannels, dx, dy = getDataset("FashionMNIST", 60000)  # FashionMNIST , MNIST

In [10]:
for inum, (batch_x, label) in enumerate(train_loader):
    break
    

In [11]:
orig = batch_x[5][0]

In [12]:
import nibabel as nib

In [19]:
image_path = '/media/chethan/New Volume/Thesis/mri_learning/mri_dwi_dataset/OASIS3/OAS30063/OAS30063_MR_d0160/dwi1/NIFTI/sub-OAS30063_ses-d0160_dwi.nii.gz'

image_obj = nib.load(image_path)

print(f'Type of the image{type(image_obj)}.')
image_data = image_obj.get_fdata()
type(image_data)

image_data[np.where(image_data < 0)] = 0
image_data = (image_data - image_data.min()) / ((image_data.max() - image_data.min()))
#print(image_data.mean())
#image_data[np.where(image_data < image_data.mean()+image_data.std())] = 0
#image_data = image_data / image_data.max()

cleaned_image = image_data[:,:,40,0]
cleaned_image.shape

orig = cleaned_image

Type of the image<class 'nibabel.nifti1.Nifti1Image'>.


In [14]:
import numpy as np
import torch
import sys

import scipy
import scipy.integrate
#sys.path.insert(1, '/home/suarez08/PhD_PINNs/PIPS_framework')
from jmp_solver.sobolev import Sobolev
from jmp_solver.sobolev import Sobolev
from jmp_solver.solver import Solver
from jmp_solver.utils import matmul
import jmp_solver.surrogates
import time
#sys.path.insert(1, '/home/suarez08/minterpy/src')
import minterpy as mp
from jmp_solver.diffeomorphisms import hyper_rect
import matplotlib
import matplotlib.pyplot as plt
#style.use('dark_background')
matplotlib.rcdefaults() 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)

In [15]:
deg_Quads = [40, 50, 60, 65, 70, 80]

all_direct_recons_psnr = []
all_noise1_recons_psnr = []
all_noise2_recons_psnr = []
all_noise4p3_recons_psnr = []

for deg_quad in deg_Quads:
    #deg_quad = 7
    rect = rect = np.array([[-1.0, 1.0], [-1.0, 1.0]])
    diffeo_param = hyper_rect(*rect)
    sob_param = Sobolev(deg=deg_quad, dim=2)
    ##### Sobolev norm for the inteprolation
    sob_param.set_s(0)
    #####
    x_plt, _, _, x, _, _ = sob_param.get_quad()
    metric_param = sob_param.metric()
    W_param = sob_param.get_leja_weights()
    u_ob = jmp_solver.surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
    metric_2d = sob_param.metric(weak=True)
    x_l = sob_param.get_xs()
    X_p = u_ob.data_axes([x,x]).T

    u_ob = jmp_solver.surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
    x = np.linspace(-1,1,96)
    X_p = u_ob.data_axes([x,x]).T

    def get_all_thetas(listedImage):
        #print('listedImage.shape',listedImage.shape)
        Fr = torch.tensor(listedImage).reshape(96*96)

        def grad_x(t,theta):
            theta_t = torch.tensor(theta)
            return -2*torch.matmul(X_p.T,(torch.matmul(X_p,theta_t)-Fr)).detach().numpy()

        def give_theta_t():
            start = time.time()
            u_ob.set_weights_val(0.0)
            theta_0 =  list(u_ob.parameters())[0][0]
            dt = 0.01
            theta_t = theta_0
            for k in range(20):
                theta_int =  scipy.integrate.RK45(grad_x, 0.1, theta_t.detach().numpy(), 100)
                theta_int.step()
                theta_t = torch.tensor(theta_int.y)
            return theta_t

        act_theta = give_theta_t()
        return act_theta

    testRK = get_all_thetas(orig)
    testRK = testRK.float()
    recIM = torch.matmul(X_p.float(), testRK.T).T
    recIM[np.where(recIM < 0.0)] = 0
    recIM = recIM.reshape(96,96)

    # PSNR of direct backward reconstruction of coefficients without perturbation 
    orig_normal = Normalize()(orig)
    recIM_norm = Normalize()(recIM)
    direct_recon_psnr = psnr(orig_normal, recIM_norm, data_range=1.)
    
    all_direct_recons_psnr.append(direct_recon_psnr)

    # MSE of direct reconstruction

    direct_recon_mse = np.mean(((orig_normal - np.array(recIM_norm))**2)*0.5)

    prozs = [0.01, 0.02, 0.0423, 0.7] 


    rand_perturb = []

    testRK_pert = np.array(testRK)
    testRK_pert = testRK_pert.reshape(1,testRK_pert.shape[0])
    for proz in prozs:
        
        rand_perturb.append(np.random.rand(1,testRK_pert.shape[1])*(np.max(testRK_pert)-np.min(testRK_pert))*proz)

    orig_perturb = []
    for rand_transform in rand_perturb:
        orig_perturb.append(torch.from_numpy(np.add(testRK_pert,rand_transform)).reshape(rand_transform.shape))#.to(device))
        
    pert_coeff = torch.tensor(orig_perturb[0]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    


    orig_normal = Normalize()(orig)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise1_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    
    all_noise1_recons_psnr.append(noise1_psnr)

    pert_coeff = torch.tensor(orig_perturb[1]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise2_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise2_recons_psnr.append(noise2_psnr)

    pert_coeff = torch.tensor(orig_perturb[2]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)

    recIM_pert10_norm = Normalize()(recIM_pert10)
    noise4p3_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise4p3_recons_psnr.append(noise4p3_psnr)

torch.save(all_direct_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/RK_MRI_direct_recon_psnr.pt')
torch.save(all_noise1_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/RK_MRI_all_noise1_recons_psnr.pt')
torch.save(all_noise2_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/RK_MRI_all_noise2_recons_psnr.pt')
torch.save(all_noise4p3_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/RK_MRI_all_noise4p3_recons_psnr.pt')

/tmp/ipykernel_22468/3591987762.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[0]).float()
/tmp/ipykernel_22468/3591987762.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[1]).float()
/tmp/ipykernel_22468/3591987762.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[2]).float()


In [16]:
all_direct_recons_psnr

[21.54758601396036,
 23.979189870014515,
 25.767243442431337,
 26.32213515750834,
 27.545704028638003,
 29.305213575968093]

In [17]:
# same for Analytical



#orig = batch_x[5][0]

In [20]:
deg_Quads = [40, 50, 60, 65, 70, 80]

all_direct_recons_psnr = []
all_noise1_recons_psnr = []
all_noise2_recons_psnr = []
all_noise4p3_recons_psnr = []

for deg_quad in deg_Quads:
    #deg_quad = 7
    #rect = rect = np.array([[-1.0, 1.0], [-1.0, 1.0]])
    #diffeo_param = hyper_rect(*rect)
    #sob_param = Sobolev(deg=deg_quad, dim=2)
    ##### Sobolev norm for the inteprolation
    #sob_param.set_s(0)
    #####
    #x_plt, _, _, x, _, _ = sob_param.get_quad()
    #metric_param = sob_param.metric()
    #W_param = sob_param.get_leja_weights()
    #u_ob = jmp_solver.surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
    #metric_2d = sob_param.metric(weak=True)
    #x_l = sob_param.get_xs()
    #X_p = u_ob.data_axes([x,x]).T

    u_ob = jmp_solver.surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
    x = np.linspace(-1,1,96)
    X_p = u_ob.data_axes([x,x]).T

    def get_all_thetas(listedImage):
        Fr = torch.tensor(listedImage).reshape(96*96).double()
        KsK = matmul(X_p.T, X_p)
        Ksf = matmul(X_p.T, Fr)
        w = matmul(KsK.inverse(), Ksf)

        act_theta = w
        return act_theta

    testRK = get_all_thetas(orig)
    testRK = testRK.float()
    recIM = torch.matmul(X_p.float(), testRK.T).T
    recIM[np.where(recIM < 0.0)] = 0
    recIM = recIM.reshape(96,96)

    # PSNR of direct backward reconstruction of coefficients without perturbation 
    orig_normal = Normalize()(orig)
    recIM_norm = Normalize()(recIM)
    direct_recon_psnr = psnr(orig_normal, recIM_norm, data_range=1.)
    
    all_direct_recons_psnr.append(direct_recon_psnr)

    # MSE of direct reconstruction

    direct_recon_mse = np.mean(((orig_normal - np.array(recIM_norm))**2)*0.5)

    prozs = [0.01, 0.02, 0.0423, 0.7] 


    rand_perturb = []

    testRK_pert = np.array(testRK)
    testRK_pert = testRK_pert.reshape(1,testRK_pert.shape[0])
    for proz in prozs:
        
        rand_perturb.append(np.random.rand(1,testRK_pert.shape[1])*(np.max(testRK_pert)-np.min(testRK_pert))*proz)

    orig_perturb = []
    for rand_transform in rand_perturb:
        orig_perturb.append(torch.from_numpy(np.add(testRK_pert,rand_transform)).reshape(rand_transform.shape))#.to(device))
        
    pert_coeff = torch.tensor(orig_perturb[0]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    


    orig_normal = Normalize()(orig)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise1_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    
    all_noise1_recons_psnr.append(noise1_psnr)

    pert_coeff = torch.tensor(orig_perturb[1]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise2_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise2_recons_psnr.append(noise2_psnr)

    pert_coeff = torch.tensor(orig_perturb[2]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)

    recIM_pert10_norm = Normalize()(recIM_pert10)
    noise4p3_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise4p3_recons_psnr.append(noise4p3_psnr)

torch.save(all_direct_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/Analytic_MRI_direct_recon_psnr.pt')
torch.save(all_noise1_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/Analytic_MRI_all_noise1_recons_psnr.pt')
torch.save(all_noise2_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/Analytic_MRI_all_noise2_recons_psnr.pt')
torch.save(all_noise4p3_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/Analytic_MRI_all_noise4p3_recons_psnr.pt')

/tmp/ipykernel_22468/4062282975.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[0]).float()
/tmp/ipykernel_22468/4062282975.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[1]).float()
/tmp/ipykernel_22468/4062282975.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[2]).float()


In [21]:
all_noise4p3_recons_psnr

[15.256353784651166,
 15.16826797604606,
 15.17076498995149,
 15.16407096354799,
 15.157777386252743,
 15.179097292651026]

In [22]:
all_direct_recons_psnr

[21.689460848850164,
 24.65499565315675,
 12.492747567899567,
 13.558073263348136,
 13.269721052914951,
 14.537121518921875]

In [23]:
# Next is LSTQS

deg_Quads = [40, 50, 60, 65, 70, 80]

all_direct_recons_psnr = []
all_noise1_recons_psnr = []
all_noise2_recons_psnr = []
all_noise4p3_recons_psnr = []

for deg_quad in deg_Quads:
    #deg_quad = 7
    #rect = rect = np.array([[-1.0, 1.0], [-1.0, 1.0]])
    #diffeo_param = hyper_rect(*rect)
    #sob_param = Sobolev(deg=deg_quad, dim=2)
    ##### Sobolev norm for the inteprolation
    #sob_param.set_s(0)
    #####
    #x_plt, _, _, x, _, _ = sob_param.get_quad()
    #metric_param = sob_param.metric()
    #W_param = sob_param.get_leja_weights()
    #u_ob = jmp_solver.surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
    #metric_2d = sob_param.metric(weak=True)
    #x_l = sob_param.get_xs()
    #X_p = u_ob.data_axes([x,x]).T

    u_ob = jmp_solver.surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
    x = np.linspace(-1,1,96)
    X_p = u_ob.data_axes([x,x]).T

    def get_all_thetas(listedImage):

        get = np.linalg.lstsq(np.array(X_p), listedImage.reshape(96*96), rcond='warn')
        act_theta = torch.tensor(get[0])
        return act_theta

    testRK = get_all_thetas(orig)
    testRK = testRK.float()
    recIM = torch.matmul(X_p.float(), testRK.T).T
    recIM[np.where(recIM < 0.0)] = 0
    recIM = recIM.reshape(96,96)

    # PSNR of direct backward reconstruction of coefficients without perturbation 
    orig_normal = Normalize()(orig)
    recIM_norm = Normalize()(recIM)
    direct_recon_psnr = psnr(orig_normal, recIM_norm, data_range=1.)
    
    all_direct_recons_psnr.append(direct_recon_psnr)

    # MSE of direct reconstruction

    direct_recon_mse = np.mean(((orig_normal - np.array(recIM_norm))**2)*0.5)

    prozs = [0.01, 0.02, 0.0423, 0.7] 


    rand_perturb = []

    testRK_pert = np.array(testRK)
    testRK_pert = testRK_pert.reshape(1,testRK_pert.shape[0])
    for proz in prozs:
        
        rand_perturb.append(np.random.rand(1,testRK_pert.shape[1])*(np.max(testRK_pert)-np.min(testRK_pert))*proz)

    orig_perturb = []
    for rand_transform in rand_perturb:
        orig_perturb.append(torch.from_numpy(np.add(testRK_pert,rand_transform)).reshape(rand_transform.shape))#.to(device))
        
    pert_coeff = torch.tensor(orig_perturb[0]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    


    orig_normal = Normalize()(orig)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise1_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    
    all_noise1_recons_psnr.append(noise1_psnr)

    pert_coeff = torch.tensor(orig_perturb[1]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise2_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise2_recons_psnr.append(noise2_psnr)

    pert_coeff = torch.tensor(orig_perturb[2]).float()
    recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)

    recIM_pert10_norm = Normalize()(recIM_pert10)
    noise4p3_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise4p3_recons_psnr.append(noise4p3_psnr)

torch.save(all_direct_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/LSTQS_MRI_direct_recon_psnr.pt')
torch.save(all_noise1_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/LSTQS_MRI_all_noise1_recons_psnr.pt')
torch.save(all_noise2_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/LSTQS_MRI_all_noise2_recons_psnr.pt')
torch.save(all_noise4p3_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/LSTQS_MRI_all_noise4p3_recons_psnr.pt')

/tmp/ipykernel_22468/3788239765.py:32: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  get = np.linalg.lstsq(np.array(X_p), listedImage.reshape(96*96), rcond='warn')
/tmp/ipykernel_22468/3788239765.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[0]).float()
/tmp/ipykernel_22468/3788239765.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[1]).float()
/tmp/ipykernel_22468/3

In [24]:
im_len = orig.shape[0]
im_len
def getLejaOrderedImage(im2d, Q_exact):
    Q_exact_sorted = (Q_exact)
    leg_pts_image_sorted = torch.tensor([])
    for i in Q_exact_sorted:
        for j in Q_exact_sorted:
            i_int = i.astype(int)
            j_int = j.astype(int)
            if(i_int == 0 or i_int == (im_len-1) or j_int == 0 or j_int == (im_len-1) ):
                req_value = im2d[i_int][j_int]
            else:
                x_co_1 = i_int
                x_co_2 = i_int+1
                y_co_1 =j_int
                y_co_2 = j_int+1

                Q_ev_11 = im2d[x_co_1][y_co_1]
                Q_ev_12 = im2d[x_co_1][y_co_2]
                Q_ev_21 = im2d[x_co_2][y_co_1]
                Q_ev_22 = im2d[x_co_2][y_co_2]

                x_co = i
                y_co = j

                term_1 = ( ( ( (x_co_2 - x_co) * (y_co_2 - y_co) ) / ( (x_co_2 - x_co_1) * (y_co_2 - y_co_1) ) ) * Q_ev_11)
                term_2 = (( ( (x_co - x_co_1) * (y_co_2 - y_co) ) / ( (x_co_2 - x_co_1) * (y_co_2 - y_co_1) ) ) * Q_ev_21)
                term_3 = ( ( ( (x_co_2 - x_co) * (y_co - y_co_1) ) / ( (x_co_2 - x_co_1) * (y_co_2 - y_co_1) ) ) * Q_ev_12)
                term_4 = ( ( ( (x_co - x_co_1) * (y_co - y_co_1) ) / ( (x_co_2 - x_co_1) * (y_co_2 - y_co_1) ) ) * Q_ev_22)

                req_value = term_1 + term_2 + term_3 + term_4                
            Fr1 = torch.tensor([req_value])
            leg_pts_image_sorted = torch.cat((leg_pts_image_sorted, Fr1)) 
    return leg_pts_image_sorted

In [25]:
# Solve the analytic gradient flow, the trained model is u_ob

def getImageFromCoefficients(imageBilinearEncoded,X_p ):
    Fr = imageBilinearEncoded
    Fr = torch.tensor(Fr)
    start = time.time()
    K = torch.eye(len(X_p))
    KsK = matmul(X_p.T, metric_2d(matmul(torch.diag(W_param),X_p)))
    Ksf = matmul(X_p.T, metric_2d(matmul(torch.diag(W_param),Fr)))
    w = matmul(KsK.inverse(), Ksf)
    u_ob.set_weights(w)
    end = time.time()
    #print('time consumption: %.2fs' % (end-start))

    Ksf = matmul(X_p.T, metric_2d(matmul(torch.diag(W_param),Fr)))
    w = matmul(KsK.inverse(), Ksf)
    u_ob.set_weights(w)


    b = np.linspace(-1,1,im_len)#np.array([x[0]])#np.linspace(-1,1,100)
    xf= np.linspace(-1,1,im_len)#x#np.linspace(-1,1,100)
    BF, XF = np.meshgrid(b,xf)

    X_test = u_ob.data_axes([b,xf]).T
    #print('X_test.shape',X_test.shape)
    X_final = u_ob.data_axes([x,x]).T
    pred = u_ob(X_test).T[0].reshape(len(b),len(xf)).detach().numpy()
    GT = Fr.reshape(len(x),len(x))
    pred[np.where(pred < 0.0)] = 0
    recIM = pred.reshape(im_len,im_len)

    return recIM

In [26]:
# Next is Sobolev Fitting


deg_Quads = [40, 50, 60, 65, 70, 80]

all_direct_recons_psnr = []
all_noise1_recons_psnr = []
all_noise2_recons_psnr = []
all_noise4p3_recons_psnr = []

for deg_quad in deg_Quads:
    #deg_quad = 23
    rect = rect = np.array([[-1.0, 1.0], [-1.0, 1.0]])
    #print(rect.shape)
    #diffeo_param = hyper_rect(*rect)
    #print('diffeo_param',diffeo_param)
    sob_param = Sobolev(deg=deg_quad, dim=2)
    #print('sob_param',sob_param)

    ##### Sobolev norm for the inteprolation
    sob_param.set_s(0)
    #####
    x_plt, _, _, x, _, _ = sob_param.get_quad()
    metric_param = sob_param.metric()
    W_param = sob_param.get_leja_weights()
    u_ob = jmp_solver.surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
    metric_2d = sob_param.metric(weak=True)
    x_l = sob_param.get_xs()
    X_p = u_ob.data_axes([x,x]).T
    #X_p = X_p.float()

    Q_exact = im_len/2 *(x+1)

    testRK = getLejaOrderedImage(orig, Q_exact)
    testRK = testRK.double()
    recIM = getImageFromCoefficients(testRK,X_p )
    recIM[np.where(recIM < 0.0)] = 0
    recIM = recIM.reshape(96,96)

    # PSNR of direct backward reconstruction of coefficients without perturbation 
    orig_normal = Normalize()(orig)
    recIM_norm = Normalize()(recIM)
    direct_recon_psnr = psnr(orig_normal, recIM_norm, data_range=1.)
    
    all_direct_recons_psnr.append(direct_recon_psnr)

    # MSE of direct reconstruction

    direct_recon_mse = np.mean(((orig_normal - np.array(recIM_norm))**2)*0.5)

    prozs = [0.01, 0.02, 0.0423, 0.7] 


    rand_perturb = []

    testRK_pert = np.array(testRK)
    testRK_pert = testRK_pert.reshape(1,testRK_pert.shape[0])
    for proz in prozs:
        
        rand_perturb.append(np.random.rand(1,testRK_pert.shape[1])*(np.max(testRK_pert)-np.min(testRK_pert))*proz)

    orig_perturb = []
    for rand_transform in rand_perturb:
        orig_perturb.append(torch.from_numpy(np.add(testRK_pert,rand_transform)).reshape(rand_transform.shape))#.to(device))
        
    pert_coeff = torch.tensor(orig_perturb[0]).float()
    #recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    pert_coeff = pert_coeff.double()
    #print(pert_coeff.shape)


    recIM_pert10 = getImageFromCoefficients(pert_coeff[0],X_p )

    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    


    orig_normal = Normalize()(orig)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise1_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    
    all_noise1_recons_psnr.append(noise1_psnr)

    pert_coeff = torch.tensor(orig_perturb[1])#.float()
    #recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T
    pert_coeff = pert_coeff.double()

    recIM_pert10 = getImageFromCoefficients(pert_coeff[0],X_p )

    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)
    recIM_pert10_norm = Normalize()(recIM_pert10)

    noise2_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise2_recons_psnr.append(noise2_psnr)

    pert_coeff = torch.tensor(orig_perturb[2])#.float()
    #recIM_pert10 = torch.matmul(X_p.float(), pert_coeff.T).T

    pert_coeff = pert_coeff.double()
    recIM_pert10 = getImageFromCoefficients(pert_coeff[0],X_p )

    recIM_pert10[np.where(recIM_pert10 < 0.0)] = 0
    recIM_pert10 = recIM_pert10.reshape(96,96)

    recIM_pert10_norm = Normalize()(recIM_pert10)
    noise4p3_psnr = psnr(orig_normal, recIM_pert10_norm, data_range=1.)
    all_noise4p3_recons_psnr.append(noise4p3_psnr)

torch.save(all_direct_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/sobolev_MRI_direct_recon_psnr.pt')
torch.save(all_noise1_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/sobolev_MRI_all_noise1_recons_psnr.pt')
torch.save(all_noise2_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/sobolev_MRI_all_noise2_recons_psnr.pt')
torch.save(all_noise4p3_recons_psnr, '/media/chethan/New Volume/Thesis/regressionExperiments/sobolev_MRI_all_noise4p3_recons_psnr.pt')

/tmp/ipykernel_22468/1353063009.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Fr = torch.tensor(Fr)
/tmp/ipykernel_22468/3650242546.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[0]).float()
/tmp/ipykernel_22468/3650242546.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pert_coeff = torch.tensor(orig_perturb[1])#.float()
/tmp/ipykernel_22468/3650242546.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().d

In [27]:
all_direct_recons_psnr

[19.346862548625474,
 20.875955749899923,
 22.109724921521966,
 22.378746573547485,
 22.235325639670037,
 23.24782391902946]

In [28]:
all_noise4p3_recons_psnr

[18.952330844273423,
 20.47516533159505,
 21.81634886014887,
 21.95561385535821,
 21.573355662292812,
 22.65224303710223]